In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#data = pd.read_csv(os.path.join('..','data','experiment_data.csv'))
data = pd.read_csv('/Users/nhalliwe/Downloads/experiment_data.csv')

In [3]:
best = 'Very intuitive, an explanation I could give or expect'
second_best = 'Intuitive'
middle = 'Neither intuitive or unintuitive'
second_worst = 'Unintuitive'
worst = 'Not intuitive at all, not an explanation I would give or expect'

data = data.replace(best,5).replace(second_best,4).replace(middle,3).replace(second_worst,2).replace(worst,1)

In [4]:
cols = [col for col in data.columns if 
        (col != 'Timestamp') and ('Optional' not in col) 
                and (not col.startswith('Do') and (not col.startswith('What'))
                and (not col.startswith('Any')) and (not col.startswith('Unnamed')))]

#drop great grandparent
cols = cols[4:]

In [5]:
clean_data = data.loc[:,cols]

In [6]:
#(clean_data/5).mean(axis=0)

In [7]:
import utils

In [8]:
rule_file = 'spouse'
MAX_PADDING = 3

In [69]:
triples,traces, weights = utils.parse_ttl(
    file_name=os.path.join('..','data','traces',rule_file+'.ttl'),
    max_padding=MAX_PADDING
)

In [100]:
_, unique_idx = np.unique(traces, axis=0,return_index=True)


In [101]:
traces[unique_idx]

array([[['<http://example.org/data#BrotherInLawPaul>', 'spouse',
         '<http://example.org/data#SisterPaul>'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT']],

       [['<http://example.org/data#BrotherPaul>', 'spouse',
         '<http://example.org/data#SisterInLawPaul>'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT']],

       [['<http://example.org/data#FatherPaul>', 'child',
         '<http://example.org/data#BrotherPaul>'],
        ['<http://example.org/data#BrotherPaul>', 'parent',
         '<http://example.org/data#MotherPaul>'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT']],

       [['<http://example.org/data#FatherPaul>', 'child',
         '<http://example.org/data#Paul>'],
        ['<http://example.org/data#Paul>', 'parent',
         '<http://example.org/data#MotherPaul>'],
        ['UNK_ENT', 'UNK_REL', 'UNK_ENT']],

       [['<http://example.org/data#FatherPaul>', 'child',
         '<http://exam